Now we try dim-red using PCA

In [1]:
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import itertools
from tabulate import tabulate

In [2]:
#Same function as in exp_0 but with a dim-red step
def pca_kmeans_classify(X, exclusions, ncomponents, nclasses):
    #Perform dim-red
    pca = PCA(n_components=ncomponents)
    Xnew = pca.fit_transform(X);
    
    #Rest is as before
    kmeans = KMeans(n_clusters=nclasses, tol=1e-6, n_init=30).fit(X)
    indx = kmeans.labels_
    
    cls_size = np.unique(indx, return_counts=True)[1]
    sorted_indx = np.argsort(cls_size)

    l=[]
    for i in sorted_indx:
        ltemp = list(heroes[np.argwhere(indx==i)].flatten())
        ltemp.sort()
        l.append(ltemp)
    ltrans = list(map(list,itertools.zip_longest(*l,fillvalue=None)))
    print(tabulate(ltrans))


    header2 = np.delete(header,exclusions,0)
    stats2 = np.delete(stats,exclusions,1)

    cls_avg = []
    cls_avg = [np.append('Metric - Class#',header2)]
    j=0
    for i in sorted_indx:
        avg = np.round(np.mean(stats2[np.argwhere(indx==i).flatten()],axis=0),1)
        std = np.std(stats2[np.argwhere(indx==i).flatten()],axis=0)
        cv = np.round(std/(avg+1e-6),2)
        avg = np.append('Avg. %d'%j,avg)
        std = np.append('CV. %d'%j,cv)
        j+=1

        cls_avg.append(avg)
        cls_avg.append(std)
    cls_avg = list(map(list,zip(*cls_avg)))
    print(tabulate(cls_avg))

In [3]:
#Load data
d = np.load('hots_stats_ext.npz')
header = d['header'][1:]
heroes = d['heroes']
stats = d['stats']

In [4]:
header

array(['Games', 'Win %', 'Avg Length', 'T/D', 'Takedowns', 'Kills',
       'Deaths', 'Hero Dmg', 'Siege Dmg', 'Healing', 'Self Heal',
       'Dmg Taken', 'XP', 'DPS', 'Range', 'HP'], dtype='<U10')

In [5]:
#Normalize data
normalizer = preprocessing.Normalizer()
norm_stats = normalizer.transform(stats)  

In [6]:
#Remove Games, Win rate, Avg. game duration (superfluous) and T/D (redundant)
#For now also exclude internal stats: DPS, Range, HP
exclusions = [0,1,2,3,-1,-2,-3]
X = np.delete(norm_stats,exclusions,1)

In [7]:
#Reduce dimensions
pca = PCA(n_components=4)
Xnew = pca.fit_transform(X);

In [20]:
print(tabulate(pca.get_covariance()))

------------  ------------  -----------  ------------  ------------  ------------  ------------  ------------  ------------
 0.000571519   2.15563e-10  1.4414e-10    1.95998e-06  -7.20485e-08   1.76412e-06  -1.18959e-07   1.57979e-06   1.88335e-07
 2.15563e-10   0.000571518  5.61149e-11   1.31751e-06   9.49701e-07  -1.05567e-06   1.50971e-08   5.30392e-07   1.4151e-07
 1.4414e-10    5.61149e-11  0.000571518   4.97023e-07   2.05459e-07   1.71661e-08   1.35937e-07   6.30082e-07   7.20425e-08
 1.95998e-06   1.31751e-06  4.97023e-07   0.014204      0.013776     -0.0103968    -0.00115981    0.00212891    0.00170579
-7.20485e-08   9.49701e-07  2.05459e-07   0.013776      0.0426345    -0.0231403     0.000639253  -0.000891881   0.00414658
 1.76412e-06  -1.05567e-06  1.71661e-08  -0.0103968    -0.0231403     0.0462592    -0.00575502   -0.00465433   -0.00177102
-1.18959e-07   1.50971e-08  1.35937e-07  -0.00115981    0.000639253  -0.00575502    0.00649094    0.0109699     0.000473899
 1.57979e-06

In [26]:
Xnew.shape

(90, 4)

In [8]:
pca_kmeans_classify(X=X,exclusions=exclusions,ncomponents=4,nclasses=4)
#4 classes: no difference

---------  -----------  ----------------  -----------
Anub'arak  Alexstrasza  Alarak            Abathur
Artanis    Ana          Arthas            Azmodan
Diablo     Anduin       Blaze             Cassia
E.T.C.     Auriel       Chen              Chromie
Garrosh    Brightwing   Cho               Deathwing
Johanna    Deckard      D.Va              Falstad
Li-Ming    Kharazim     Dehaka            Fenix
Mei        Li Li        Genji             Gall
Muradin    Lt. Morales  Hogger            Gazlowe
Raynor     Lúcio        Illidan           Greymane
Sonya      Malfurion    Imperius          Gul'dan
Stitches   Rehgar       Kerrigan          Hanzo
Tychus     Stukov       Leoric            Jaina
Valla      Tyrande      Maiev             Junkrat
Varian     Uther        Mal'Ganis         Kael'thas
           Whitemane    Malthael          Kel'Thuzad
                        Medivh            Lunara
                        Qhira             Mephisto
                        Rexxar            Murky


In [9]:
pca_kmeans_classify(X=X,exclusions=exclusions,ncomponents=4,nclasses=5)
#With 5 classes, Sonya moves to class 4. Otherwise no change

---------  -----------  -----------  ---------  ----------------
Anub'arak  Alexstrasza  Abathur      Azmodan    Alarak
Artanis    Ana          Chromie      Cassia     Arthas
Diablo     Anduin       Deathwing    Falstad    Blaze
E.T.C.     Auriel       Fenix        Gazlowe    Chen
Garrosh    Brightwing   Gall         Greymane   Cho
Johanna    Deckard      Gul'dan      Hanzo      D.Va
Mei        Kharazim     Junkrat      Jaina      Dehaka
Muradin    Li Li        Kel'Thuzad   Kael'thas  Genji
Stitches   Lt. Morales  Mephisto     Li-Ming    Hogger
Varian     Lúcio        Murky        Lunara     Illidan
           Malfurion    Probius      Nazeebo    Imperius
           Rehgar       Ragnaros     Nova       Kerrigan
           Stukov       Samuro       Orphea     Leoric
           Tyrande      Sgt. Hammer  Raynor     Maiev
           Uther        Xul          Sonya      Mal'Ganis
           Whitemane    Zagara       Sylvanas   Malthael
                                     Tassadar   Medivh


In [10]:
#Try with internal stats
exclusions = [0,1,2,3]
X = np.delete(norm_stats,exclusions,1)

In [11]:
pca_kmeans_classify(X=X,exclusions=exclusions,ncomponents=4,nclasses=4)
#4 classes: no change

---------  -----------  ----------------  -----------
Anub'arak  Alexstrasza  Alarak            Abathur
Artanis    Ana          Arthas            Azmodan
Diablo     Anduin       Blaze             Cassia
E.T.C.     Auriel       Chen              Chromie
Garrosh    Brightwing   Cho               Deathwing
Johanna    Deckard      D.Va              Falstad
Li-Ming    Kharazim     Dehaka            Fenix
Mei        Li Li        Genji             Gall
Muradin    Lt. Morales  Hogger            Gazlowe
Raynor     Lúcio        Illidan           Greymane
Sonya      Malfurion    Imperius          Gul'dan
Stitches   Rehgar       Kerrigan          Hanzo
Tychus     Stukov       Leoric            Jaina
Valla      Tyrande      Maiev             Junkrat
Varian     Uther        Mal'Ganis         Kael'thas
           Whitemane    Malthael          Kel'Thuzad
                        Medivh            Lunara
                        Qhira             Mephisto
                        Rexxar            Murky


In [12]:
pca_kmeans_classify(X=X,exclusions=exclusions,ncomponents=4,nclasses=5)
#5 classes: Sonya change only

---------  -----------  -----------  ---------  ----------------
Anub'arak  Alexstrasza  Abathur      Azmodan    Alarak
Artanis    Ana          Chromie      Cassia     Arthas
Diablo     Anduin       Deathwing    Falstad    Blaze
E.T.C.     Auriel       Fenix        Gazlowe    Chen
Garrosh    Brightwing   Gall         Greymane   Cho
Johanna    Deckard      Gul'dan      Hanzo      D.Va
Mei        Kharazim     Junkrat      Jaina      Dehaka
Muradin    Li Li        Kel'Thuzad   Kael'thas  Genji
Stitches   Lt. Morales  Mephisto     Li-Ming    Hogger
Varian     Lúcio        Murky        Lunara     Illidan
           Malfurion    Probius      Nazeebo    Imperius
           Rehgar       Ragnaros     Nova       Kerrigan
           Stukov       Samuro       Orphea     Leoric
           Tyrande      Sgt. Hammer  Raynor     Maiev
           Uther        Xul          Sonya      Mal'Ganis
           Whitemane    Zagara       Sylvanas   Malthael
                                     Tassadar   Medivh


PCA isn't having a huge impact, even with internal stats added to remove some of the redundancy. 
Nest: we try autoencoders